In [21]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.metrics import silhouette_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_parquet('/Users/manu/Desktop/SUP/Projet 2/AI_model_urban_mobility/data/df_final_15min_NoNan_20250505.parquet')

# Gestion des données temporelles

Création de profils temporels : des profils regroupant des créneaux horaires similaires selon le jour, l’heure, les vacances et les jours fériés pour chaque tronçon. Ça permet de capturer les habitudes de trafic à différents moments de la semaine.

En ajoutant ces profils au modèle, on aide la prédiction à mieux comprendre comment le trafic change selon le temps et le lieu.

## Calcul des moyennes horaires par tronçon

Création d'une variable quarter_hour qui identifie chaque créneau de 15 minutes dans la journée, afin de capturer plus finement la variation temporelle du trafic dans le modèle.

In [3]:
df['quarter_hour'] = df['hour'] * 4 + df['minute'] // 15

## Touver le nombre de cluster ideal

In [4]:
def find_best_k(df_t, max_k=10):
    X = df_t[['weekday', 'quarter_hour', 'is_vacances', 'is_ferie']].copy()
    X_scaled = StandardScaler().fit_transform(X)

    best_score = -1
    best_k = 2

    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)

        if score > best_score:
            best_score = score
            best_k = k

    return best_k

In [5]:
# Application sur un echantillon de 200 tronçons aléatoire
troncons_sample = np.random.choice(df['troncon_enc'].unique(), size=200, replace=False)
k_list = []

for troncon in tqdm(troncons_sample, desc="Recherche de k optimal"):
    df_t = df[df['troncon_enc'] == troncon]
    if len(df_t) >= 10: 
        k_opt = find_best_k(df_t, max_k=10)
        k_list.append(k_opt)

# Trouver la valeur la plus fréquente
mode_k = Counter(k_list).most_common(1)[0][0]
print(f"Nombre optimal de clusters trouvé sur 200 tronçons : {mode_k}")

Recherche de k optimal:   0%|          | 0/200 [00:00<?, ?it/s]

Recherche de k optimal: 100%|██████████| 200/200 [03:27<00:00,  1.04s/it]

Nombre optimal de clusters trouvé sur 200 tronçons : 8


## Aggrégation

Réduit la granularité temporelle et avoir des profils plus stables et moins bruités, ce qui facilite le clustering et donne des clusters plus représentatifs.

In [6]:
agg = df.groupby(['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie']).agg({
    'code_couleur': 'mean'
}).reset_index()


## Application du clustering

In [7]:
def compute_final_time_clusters(df_agg, n_clusters):
    all_clusters = []

    for troncon in tqdm(df_agg['troncon_enc'].unique(), desc="Clustering final par tronçon"):
        df_t = df_agg[df_agg['troncon_enc'] == troncon].copy()
        X = df_t[['weekday', 'quarter_hour', 'is_vacances', 'is_ferie']]
        X_scaled = StandardScaler().fit_transform(X)

        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        df_t['time_cluster'] = kmeans.fit_predict(X_scaled)

        all_clusters.append(df_t)

    return pd.concat(all_clusters, ignore_index=True)


In [8]:
# Application sur les données agrégées
df_clusters = compute_final_time_clusters(agg, n_clusters=8)

Clustering final par tronçon: 100%|██████████| 749/749 [00:09<00:00, 78.95it/s]


In [9]:
# Merge sur les colonnes clés temporelles et tronçon
df_final = df.merge(
    df_clusters[['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie', 'time_cluster']],
    on=['troncon_enc', 'weekday', 'quarter_hour', 'is_vacances', 'is_ferie'],
    how='left'
)

## Vérification

In [17]:
print(len(df_final))
print(len(df))

1880739
1880739


In [18]:
# Compter le nombre de lignes par tronçon
count_per_troncon = df_final.groupby('troncon_enc').size().reset_index(name='count')
count_per_troncon.loc[count_per_troncon['count'] != 2511]

,troncon_enc,count


# Gestion des données météo 

In [23]:
meteo_vars = ['temperature_2m', 'visibility', 'precipitation', 'wind_speed_10m']

In [24]:
# Vérification valeur manquante
print(df_final[meteo_vars].isnull().sum())

temperature_2m    0
visibility        0
precipitation     0
wind_speed_10m    0
dtype: int64


In [26]:
# Vérification valeur negatives ou impossibles
for var in ['precipitation', 'wind_speed_10m', 'visibility']:
    print(f"Valeurs négatives dans {var} : {(df_final[var] < 0).sum()}")

Valeurs négatives dans precipitation : 0
Valeurs négatives dans wind_speed_10m : 0
Valeurs négatives dans visibility : 0


In [28]:
# Vérification valeur extreme
for var in meteo_vars:
    print(f"{var} min : {df_final[var].min()}, max : {df_final[var].max()}")

temperature_2m min : 0.3305, max : 18.130499
visibility min : 2160.0, max : 24140.0
precipitation min : 0.0, max : 5.1
wind_speed_10m min : 0.8049845, max : 33.127823


In [ ]:
# Vérification de la variabilité (ce n'est pas toujours la mm valeur qui revient)
for var in meteo_vars:
    print(f"{var} nombre de valeurs uniques : {df_final[var].nunique()}")

temperature_2m nombre de valeurs uniques : 270
visibility nombre de valeurs uniques : 60
precipitation nombre de valeurs uniques : 26
wind_speed_10m nombre de valeurs uniques : 446


In [ ]:
# Vérification de la présennce de pluie
for var in ['precipitation']:
    zeros = (df_final[var] == 0).sum()
    total = len(df_final)
    print(f"{var} = 0 dans {zeros} lignes soit {zeros/total:.2%}")

precipitation = 0 dans 1728692 lignes soit 91.92%


Conclusion : Les variables météo ne contiennent aucune valeur manquante, ce qui est un bon point pour la qualité des données. Les valeurs extrêmes observées sont dans des plages réalistes pour la région considérée (ex. température entre 0,33°C et 18,13°C, précipitations jusqu’à 5,1 mm sur un quart d’heure).

La précipitation est à zéro dans environ 92 % des cas, ce qui est logique car il ne pleut pas tout le temps, mais cela signifie que cette variable peut avoir un impact limité dans certains cas.

Dans l’ensemble, les données météo semblent propres et exploitables directement dans le modèle sans nettoyage particulier, mais il faudra surveiller l’impact des variables avec peu de variabilité comme la précipitation souvent nulle.

# Vérification des corrélations

In [34]:
# Calculer la corrélation de Pearson entre toutes les variables numériques et 'code_couleur'
corr_matrix = df_final.select_dtypes('number').corr()

# Extraire la corrélation avec 'code_couleur' et trier par valeur absolue décroissante
corr_with_target = corr_matrix['code_couleur'].drop('code_couleur').abs().sort_values(ascending=False)

print("Variables les plus corrélées avec code_couleur :")
print(corr_with_target)


Variables les plus corrélées avec code_couleur :
taux_occupation      0.819590
temps_de_parcours    0.282691
vitesse              0.269315
debit                0.212483
quarter_hour         0.044439
hour                 0.044235
temperature_2m       0.039239
weekday              0.032252
wind_speed_10m       0.030965
is_vacances          0.030756
troncon_enc          0.026609
code_no2             0.025660
time_cluster         0.008434
longueur             0.008411
minute               0.005995
code_pm25            0.004685
code_qual            0.004033
precipitation        0.002983
code_pm10            0.002673
visibility           0.000523
is_ferie                  NaN
code_zone                 NaN
code_so2                  NaN
code_o3                   NaN
x_wgs84                   NaN
y_wgs84                   NaN
x_reg                     NaN
y_reg                     NaN
epsg_reg                  NaN
geo_point_2d_lon          NaN
geo_point_2d_lat          NaN
Name: code_couleur, d

Les variables les plus corrélées au trafic sont nos 4 variables temporelles. Mise de côté, on s'aperçoit que quarter_hour est la variable suivante la plus corrélée, bien plus que notre variable de clustering time_cluster. Deux modèles seront testés : 1 - avec le clustering 2 - sans le clustering